# Lab: view jobs and stages in the Spark Application UI

This lab relies on a previous lab, in that it uses the /FileStore/accounts/ and /FileStore/logs/ les. You need to ensure
that you have those les in the right place rst. We will explore the stages and tasks involved in a join job.

## Explore Partitioning of le-based RDDs

--> 1. Review /FileStore/accounts/ dataset you created in a previous lab using dbutils. Take note of the number of les
 you should do this in two ways:
(a) By listing the les in a directory
(b) A Pythonic way that outputs the number of les in a directory. (Hint: len may be your friend.)

In [0]:
# To see how many files are present in the accounts directory --> Using len() 

print("Number of files in 'dbfs:/FileStore/tables/accounts' : ", len(dbutils.fs.ls("dbfs:/FileStore/tables/accounts")))

Number of files in 'dbfs:/FileStore/tables/accounts' : 7

--> 2. Create an RDD based on a single le in the dataset, e.g. /FileStore/accounts/part-m-00000, and then call toDebugString
on the RDD, which displays the number of partitions in parentheses () before the RDD id. How many partitions are
in the resulting RDD?

In [0]:
# To use toDebugString() method

myRDD = sc.textFile("dbfs:/FileStore/tables/accounts/part-m-00000")
myRDD.toDebugString()

Out[3]: b'(2) dbfs:/FileStore/tables/accounts/part-m-00000 MapPartitionsRDD[187] at textFile at NativeMethodAccessorImpl.java:0 []\n | dbfs:/FileStore/tables/accounts/part-m-00000 HadoopRDD[186] at textFile at NativeMethodAccessorImpl.java:0 []'

--> 3. Repeat this process, but specify a minimum of three partitions to sc.textFile. Does the RDD correctly have three
partitions?

In [0]:
# To use toDebugString() method

myRDD = sc.textFile("dbfs:/FileStore/tables/accounts/part-m-00000", 3)
myRDD.toDebugString()

Out[4]: b'(3) dbfs:/FileStore/tables/accounts/part-m-00000 MapPartitionsRDD[189] at textFile at NativeMethodAccessorImpl.java:0 []\n | dbfs:/FileStore/tables/accounts/part-m-00000 HadoopRDD[188] at textFile at NativeMethodAccessorImpl.java:0 []'

--> 4. Finally, create an RDD based on all the les in the accounts dataset. How does the number of les in the dataset
compare to the number of partitions in the RDD?

In [0]:
# To use toDebugString() method --> only this time we see more partitions because of more data

myRDD_all = sc.textFile("dbfs:/FileStore/tables/accounts/part*")
myRDD_all.toDebugString()

Out[5]: b'(7) dbfs:/FileStore/tables/accounts/part* MapPartitionsRDD[191] at textFile at NativeMethodAccessorImpl.java:0 []\n | dbfs:/FileStore/tables/accounts/part* HadoopRDD[190] at textFile at NativeMethodAccessorImpl.java:0 []'

## Set up the job

--> 5. Create an RDD of accounts from the contents of the /FileStore/accounts/ directory, keyed by ID (the rst eld in
the le) and with first name, last name for the value.

In [0]:
myRDD_all.take(5)

Out[6]: ['1,2008-10-23 16:05:05.0,\\N,Donald,Becton,2275 Washburn Street,Oakland,CA,94660,5100032418,2014-03-18 13:29:47.0,2014-03-18 13:29:47.0',
 '2,2008-11-12 03:00:01.0,\\N,Donna,Jones,3885 Elliott Street,San Francisco,CA,94171,4150835799,2014-03-18 13:29:47.0,2014-03-18 13:29:47.0',
 '3,2008-12-21 09:19:50.0,\\N,Dorthy,Chalmers,4073 Whaley Lane,San Mateo,CA,94479,6506877757,2014-03-18 13:29:47.0,2014-03-18 13:29:47.0',
 '4,2008-11-28 00:08:09.0,\\N,Leila,Spencer,1447 Ross Street,San Mateo,CA,94444,6503198619,2014-03-18 13:29:47.0,2014-03-18 13:29:47.0',
 '5,2008-11-15 23:06:06.0,\\N,Anita,Laughlin,2767 Hill Street,Richmond,CA,94872,5107754354,2014-03-18 13:29:47.0,2014-03-18 13:29:47.0']

In [0]:
myRDD_Users = myRDD_all.keyBy(lambda line: line.split(',')[0]).mapValues(lambda line: (line.split(',')[3] , line.split(',')[4]))
myRDD_Users.take(5)

Out[7]: [('1', ('Donald', 'Becton')),
 ('2', ('Donna', 'Jones')),
 ('3', ('Dorthy', 'Chalmers')),
 ('4', ('Leila', 'Spencer')),
 ('5', ('Anita', 'Laughlin'))]

--> 6. Construct a userreqs RDD from the data in /FileStore/logs/ directory with the total number of web hits for each
user ID.

In [0]:
logsRDD = sc.textFile("dbfs:/FileStore/tables/logs/*")
logsRDD.toDebugString()

Out[8]: b'(364) dbfs:/FileStore/tables/logs/* MapPartitionsRDD[195] at textFile at NativeMethodAccessorImpl.java:0 []\n | dbfs:/FileStore/tables/logs/* HadoopRDD[194] at textFile at NativeMethodAccessorImpl.java:0 []'

In [0]:
logsRDD.take(5)

Out[9]: ['3.94.78.5 - 69827 [15/Sep/2013:23:58:36 +0100] "GET /KBDOC-00033.html HTTP/1.0" 200 14417 "http://www.loudacre.com" "Loudacre Mobile Browser iFruit 1"',
 '3.94.78.5 - 69827 [15/Sep/2013:23:58:36 +0100] "GET /theme.css HTTP/1.0" 200 3576 "http://www.loudacre.com" "Loudacre Mobile Browser iFruit 1"',
 '19.38.140.62 - 21475 [15/Sep/2013:23:58:34 +0100] "GET /KBDOC-00277.html HTTP/1.0" 200 15517 "http://www.loudacre.com" "Loudacre Mobile Browser Ronin S1"',
 '19.38.140.62 - 21475 [15/Sep/2013:23:58:34 +0100] "GET /theme.css HTTP/1.0" 200 13353 "http://www.loudacre.com" "Loudacre Mobile Browser Ronin S1"',
 '129.133.56.105 - 2489 [15/Sep/2013:23:58:34 +0100] "GET /KBDOC-00033.html HTTP/1.0" 200 10590 "http://www.loudacre.com" "Loudacre Mobile Browser Sorrento F00L"']

In [0]:
# Total webhits for each user --> 

userreqs = logsRDD.map(lambda line: line.split()[2]).map(lambda line: (line, 1)).reduceByKey(lambda v1, v2 : v1 + v2)

userreqs.take(5)

Out[10]: [('70', 2604), ('17596', 128), ('48737', 180), ('35952', 164), ('80519', 164)]

--> 7. Join the two RDDs by user ID, and construct a new RDD based on rst name, last name and total hits.

In [0]:
Joined_RDD = myRDD_Users.join(userreqs)

Joined_RDD.take(5)

Out[11]: [('615', (('Millard', 'Tran'), 124)),
 ('1300', (('Ricky', 'Marshall'), 16)),
 ('1450', (('Hazel', 'Connolly'), 88)),
 ('4961', (('Nellie', 'Gray'), 48)),
 ('4987', (('Hosea', 'Jordan'), 196))]

In [0]:
# To make this format readable --> 

Users_Hits = Joined_RDD.map(lambda line : (line[0], line[1][0][0], line[1][0][1], line[1][1]))

Users_Hits.take(5)

Out[12]: [('615', 'Millard', 'Tran', 124),
 ('1300', 'Ricky', 'Marshall', 16),
 ('1450', 'Hazel', 'Connolly', 88),
 ('4961', 'Nellie', 'Gray', 48),
 ('4987', 'Hosea', 'Jordan', 196)]

--> 8. Print the results of accounthits.toDebugString and review the output. Based on this, see if you can determine
(a) How many stages are in this job?
(b) Which stages are dependent on which?
(c) How many tasks will each stage consist of?

In [0]:
Users_Hits.toDebugString()

Out[13]: b'(371) PythonRDD[211] at RDD at PythonRDD.scala:58 []\n | MapPartitionsRDD[208] at mapPartitions at PythonRDD.scala:183 []\n | ShuffledRDD[207] at partitionBy at NativeMethodAccessorImpl.java:0 []\n +-(371) PairwiseRDD[206] at join at <command-2319763555555278>:1 []\n | PythonRDD[205] at join at <command-2319763555555278>:1 []\n | UnionRDD[204] at union at NativeMethodAccessorImpl.java:0 []\n | PythonRDD[202] at RDD at PythonRDD.scala:58 []\n | dbfs:/FileStore/tables/accounts/part* MapPartitionsRDD[191] at textFile at NativeMethodAccessorImpl.java:0 []\n | dbfs:/FileStore/tables/accounts/part* HadoopRDD[190] at textFile at NativeMethodAccessorImpl.java:0 []\n | PythonRDD[203] at RDD at PythonRDD.scala:58 []\n | MapPartitionsRDD[200] at mapPartitions at PythonRDD.scala:183 []\n | ShuffledRDD[199] at partitionBy at NativeMethodAccessorImpl.java:0 []\n +-(364) PairwiseRDD[198] at reduceByKey at <command-2319763555555276>:3 []\n | PythonRDD[197] at reduceByKey at <command-2319763555555276>:3 []\n | dbfs:/FileStore/tables/logs/* MapPartitionsRDD[195] at textFile at NativeMethodAccessorImpl.java:0 []\n | dbfs:/FileStore/tables/logs/* HadoopRDD[194] at textFile at NativeMethodAccessorImpl.java:0 []'

In [0]:
for line in Users_Hits.toDebugString().decode().splitlines():
  print(line)

(371) PythonRDD[211] at RDD at PythonRDD.scala:58 []
 MapPartitionsRDD[208] at mapPartitions at PythonRDD.scala:183 []
 ShuffledRDD[207] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(371) PairwiseRDD[206] at join at <command-2319763555555278>:1 []
 PythonRDD[205] at join at <command-2319763555555278>:1 []
 UnionRDD[204] at union at NativeMethodAccessorImpl.java:0 []
 PythonRDD[202] at RDD at PythonRDD.scala:58 []
 dbfs:/FileStore/tables/accounts/part* MapPartitionsRDD[191] at textFile at NativeMethodAccessorImpl.java:0 []
 dbfs:/FileStore/tables/accounts/part* HadoopRDD[190] at textFile at NativeMethodAccessorImpl.java:0 []
 PythonRDD[203] at RDD at PythonRDD.scala:58 []
 MapPartitionsRDD[200] at mapPartitions at PythonRDD.scala:183 []
 ShuffledRDD[199] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(364) PairwiseRDD[198] at reduceByKey at <command-2319763555555276>:3 []
 PythonRDD[197] at reduceByKey at <command-2319763555555276>:3 []
 dbfs:/FileStore/tables/logs/* MapPartitionsRDD[195] at textFile at NativeMethodAccessorImpl.java:0 []
 dbfs:/FileStore/tables/logs/* HadoopRDD[194] at textFile at NativeMethodAccessorImpl.java:0 []

## Run the job and review it in the Spark UI

--> 9. In the information listed under the cluster's Spark UI tab, you can view the jobs tab. There shouldn't be a job running
at this point, but there may be some completed jobs of you've already run something using this cluster.

--> 12. Save the output of the RDD you created in Point 7 to /FileStore/userreqs by executing the saveAsTextFile action.

In [0]:
#Users_Hits.saveAsTextFile("dbfs:/FileStore/userreqs")   # Marked as Done

--> 13. Revisit the Spark UI jobs page. Your job will appear in the Active Jobs list until it completes, and then it will display
in the Completed Jobs List.

In [0]:
dbutils.fs.ls("dbfs:/FileStore/userreqs/")

Out[16]: [FileInfo(path='dbfs:/FileStore/userreqs/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00000', name='part-00000', size=3283),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00001', name='part-00001', size=3393),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00002', name='part-00002', size=3223),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00003', name='part-00003', size=3474),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00004', name='part-00004', size=3166),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00005', name='part-00005', size=3273),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00006', name='part-00006', size=3310),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00007', name='part-00007', size=3177),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00008', name='part-00008', size=3126),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00009', name='part-00009', size=3270),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00010', name='part-00010', size=3337),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00011', name='part-00011', size=3405),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00012', name='part-00012', size=3941),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00013', name='part-00013', size=3267),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00014', name='part-00014', size=3664),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00015', name='part-00015', size=3076),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00016', name='part-00016', size=3599),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00017', name='part-00017', size=2893),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00018', name='part-00018', size=3513),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00019', name='part-00019', size=3807),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00020', name='part-00020', size=3273),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00021', name='part-00021', size=3941),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00022', name='part-00022', size=3075),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00023', name='part-00023', size=2837),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00024', name='part-00024', size=3187),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00025', name='part-00025', size=3263),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00026', name='part-00026', size=3480),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00027', name='part-00027', size=2650),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00028', name='part-00028', size=4377),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00029', name='part-00029', size=3267),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00030', name='part-00030', size=3421),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00031', name='part-00031', size=3460),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00032', name='part-00032', size=3017),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00033', name='part-00033', size=3646),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00034', name='part-00034', size=3032),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00035', name='part-00035', size=3239),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00036', name='part-00036', size=3669),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00037', name='part-00037', size=3013),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00038', name='part-00038', size=2927),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00039', name='part-00039', size=3213),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00040', name='part-00040', size=2821),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00041', name='part-00041', size=3656),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00042', name='part-00042', size=2640),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00043', name='part-00043', size=3067),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00044', name='part-00044', size=2933),
 FileInfo(path='dbfs:/FileStore/userreqs/part-00045', name='part-00045', size=3731),
 Fi

--> 13. Click on the job description (which is the last action in the job) to see the stages. As the job progresses you want to
refresh the page a few times.
Things to note:
(a) How many stages are in the job? Does it match the number you expected from the RDD's toDebugString output?
(b) Note the times the stages were submitted to determine the order. Does the order match what you expected based
on RDD dependency?
(c) How many tasks are in each stage? The number of tasks in the rst stages correspond to the number of partitions,
which for this example corresponds to the number of les processed.
(d) The Shue Read and Shue Write columns indicate how much data was copied between tasks. This is useful to
know because copying too much data across the network can cause performance issues.

--> Challenge: dierence between reduceByKey and groupByKey
In this optional challenge part, we'll take a look at the timing dierences between reduceByKey, groupByKey and countByKey.

In [0]:
pass
# reduceByKey - Transformation - grouping + aggregating
# groupByKey - Transformation - grouping only --> (key, memory_value)  --> ( 'xyz', 002000340023403204xdawe23<>  )
# countByKey - action

--> 18. Create an RDD using paralelize which consists of the numbers 0 to 1000000 (those can be accessed using Python's
range function).

In [0]:
mil_RDD = sc.parallelize(range(1000000))
mil_RDD.toDebugString()

Out[18]: b'(8) PythonRDD[213] at RDD at PythonRDD.scala:58 []\n | ParallelCollectionRDD[212] at readRDDFromInputStream at PythonRDD.scala:413 []'

--> 19. All those records are distinct, and not in the pair RDD format. Since we're wanting to compare pair RDDs with
multiple occurences of the same key, we'll use the modulo % operation to give us RDDs with some nice repeats as
follows:

In [0]:
repeatsRDD = mil_RDD.map(lambda x: (x % 5000 , 1))
repeatsRDD.take(20)

Out[19]: [(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1)]

--> 20. You should examine the RDD you've just created (you could rst look at one with x % 5 to see the form of the created
RDD).

In [0]:
# Already did

--> 21. You should use the Spark UI to examine the running time of each of reduceByKey and groupByKey used to count /
group the number of occurrences of all the keys.

In [0]:
groupRDD = repeatsRDD.groupByKey()

for tupl in groupRDD.take(10):
    print(tupl[0], ":", len([val for val in tupl[1]]))

0 : 200
8 : 200
16 : 200
24 : 200
32 : 200
40 : 200
48 : 200
56 : 200
64 : 200
72 : 200

In [0]:
reduceRDD = repeatsRDD.reduceByKey(lambda v1,v2 : v1+v2)

reduceRDD.take(5)

Out[22]: [(0, 200), (8, 200), (16, 200), (24, 200), (32, 200)]

In [0]:
myRDD=Users_Hits.filter(lambda x: x[3]>5)
myRDD.count()

Out[32]: 31342

In [0]:
Users_Hits.count()

Out[33]: 36032

In [0]:
from pyspark import StorageLevel
myRDD.persist(StorageLevel.MEMORY_ONLY)

Out[29]: PythonRDD[228] at RDD at PythonRDD.scala:58

In [0]:
myRDD.cache()

Out[34]: PythonRDD[231] at RDD at PythonRDD.scala:58

In [0]:
myRDD.count()

Out[35]: 31342

In [0]:
myRDD.count()

Out[36]: 31342

In [0]:
myRDD.toDebugString().decode().splitlines()


Out[38]: ['(371) PythonRDD[231] at RDD at PythonRDD.scala:58 [Memory Serialized 1x Replicated]',
 ' | CachedPartitions: 371; MemorySize: 794.1 KiB; ExternalBlockStoreSize: 0.0 B; DiskSize: 0.0 B',
 ' | MapPartitionsRDD[208] at mapPartitions at PythonRDD.scala:183 [Memory Serialized 1x Replicated]',
 ' | ShuffledRDD[207] at partitionBy at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]',
 ' +-(371) PairwiseRDD[206] at join at <command-2319763555555278>:1 [Memory Serialized 1x Replicated]',
 ' | PythonRDD[205] at join at <command-2319763555555278>:1 [Memory Serialized 1x Replicated]',
 ' | UnionRDD[204] at union at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]',
 ' | PythonRDD[202] at RDD at PythonRDD.scala:58 [Memory Serialized 1x Replicated]',
 ' | dbfs:/FileStore/tables/accounts/part* MapPartitionsRDD[191] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]',
 ' | dbfs:/FileStore/tables/accounts/part* HadoopRDD[190] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]',
 ' | PythonRDD[203] at RDD at PythonRDD.scala:58 [Memory Serialized 1x Replicated]',
 ' | MapPartitionsRDD[200] at mapPartitions at PythonRDD.scala:183 [Memory Serialized 1x Replicated]',
 ' | ShuffledRDD[199] at partitionBy at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]',
 ' +-(364) PairwiseRDD[198] at reduceByKey at <command-2319763555555276>:3 [Memory Serialized 1x Replicated]',
 ' | PythonRDD[197] at reduceByKey at <command-2319763555555276>:3 [Memory Serialized 1x Replicated]',
 ' | dbfs:/FileStore/tables/logs/* MapPartitionsRDD[195] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]',
 ' | dbfs:/FileStore/tables/logs/* HadoopRDD[194] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]']